In [1]:
library(tidyverse)
library(glue)
library(stringr)
library(ggplotify)
library(janitor)
library(gridExtra)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘glue’ was built under R version 4.1.2”
Warning message:
“package ‘ggplotify’ was built under R version 4.1.3”

Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test



Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine




# Load Data from Supplementary Table 6

In [1]:
readxl::excel_sheets("../../../data/manuscript/ST6.xlsx")

[1] "README"                  "ST6-1-Module-Genes"     
[3] "ST6-2-All-Enrichment"    "ST6-3-Cell-Enrichment"  
[5] "ST6-4-Common-Enrichment" "ST6-5-Rare-Enrichment"  
[7] "ST6-6-Module-Overlap"

In [3]:
big_mod_tbl <- readxl::read_excel("../../../data/manuscript/ST6.xlsx", sheet = "ST6-1-Module-Genes")
enrich_tbl <- readxl::read_excel("../../../data/manuscript/ST6.xlsx", sheet = "ST6-2-All-Enrichment")
overlap_tbl <- readxl::read_excel("../../../data/manuscript/ST6.xlsx", sheet = "ST6-6-Module-Overlap")

In [4]:
enrich_tbl <- enrich_tbl %>%
                mutate(fdr = p.adjust(p, "fdr")) %>%
                mutate(sig = if_else(fdr <= 0.05, 1, 0))

In [5]:
all_module_ids <- stringr::str_sort(big_mod_tbl$assigned_module %>% unique,
                                    numeric = TRUE)
all_module_ids

[1] "M1"   "M2"   "M3"   "M4"   "M5"   "M6"   "M7"   "M8"   "M9"   "M10" 
 [11] "M11"  "M12"  "M13"  "M14"  "M15"  "M16"  "M17"  "M18"  "M19"  "M20" 
 [21] "M21"  "M22"  "M23"  "M24"  "M25"  "M26"  "M27"  "M28"  "M29"  "M30" 
 [31] "M31"  "M32"  "M33"  "M34"  "M35"  "M36"  "M37"  "M38"  "M39"  "M40" 
 [41] "M41"  "M42"  "M43"  "M44"  "M45"  "M46"  "M47"  "M48"  "M49"  "M50" 
 [51] "M51"  "M52"  "M53"  "M54"  "M55"  "M56"  "M57"  "M58"  "M59"  "M60" 
 [61] "M61"  "M62"  "M63"  "M64"  "M65"  "M66"  "M67"  "M68"  "M69"  "M70" 
 [71] "M71"  "M72"  "M73"  "M74"  "M75"  "M76"  "M77"  "M78"  "M79"  "M80" 
 [81] "M81"  "M82"  "M83"  "M84"  "M85"  "M86"  "M87"  "M88"  "M89"  "M90" 
 [91] "M91"  "M92"  "M93"  "M94"  "M95"  "M96"  "M97"  "M98"  "M99"  "M100"
[101] "M101" "M102" "M103" "M104" "M105" "M106" "M107" "M108" "M109" "M110"
[111] "M111" "M112" "M113" "M114" "M115" "M116" "M117" "M118" "M119" "M120"
[121] "M121" "M122" "M123" "M124"

In [6]:
wgcna_color_tbl <- big_mod_tbl %>%
    select(assigned_module, full_dendro_name, classic_name) %>%
    group_by(assigned_module) %>%
    slice_head(n = 1) %>%
    rowwise %>%
    mutate(wgcna_color = str_match(classic_name, "[^_]+_[^_]+_([^_]*)")[2])

head(wgcna_color_tbl, 3)

assigned_module,full_dendro_name,classic_name,wgcna_color
<chr>,<chr>,<chr>,<chr>
M1,M1-tr2,tr2_gene_lightyellow,lightyellow
M10,M10-iso,dev_iso_pink,pink
M100,M100-xx,xx_gene_yellow,yellow


In [7]:
rare_var_list <- readRDS("../../../data/accessory_files/rare_variant_list.rds")
rare_var_tbl <- do.call(bind_rows, rare_var_list)
rare_var_top_genes <- rare_var_tbl %>%
                        group_by(dataset) %>%
                        mutate(p = 10^-p_log) %>%
                        mutate(fdr = p.adjust(p, "fdr")) %>%
                        ungroup %>%
                        filter(is.na(fdr) | fdr <= 0.05) %>%
                        select(dataset, gene_id, gene_symbol)
table(rare_var_top_genes$dataset)


    ASD_Fu2022_TADA    ASD_SFARI_CAT1&2 ASD_SFARI_SYNDROMIC      DD_Fu2022_TADA 
                177                 895                 153                 450 
       DDD_DECIPHER              EPI_25               RUZZO          SCZ_SCHEMA 
               1269                   2                 116                  32 

In [8]:
options(warn=-1)

# Background for GO term enrichment
# Set of all genes with kME greater than 0 with any module
go_bg <- big_mod_tbl %>%
                filter(kME > 0) %>%
                select(gene_id) %>%
                pull %>%
                unique

for (module_id in all_module_ids) {
    ################
    ### Preamble ###
    ################
    
    print(glue("[INFO] Generating module dashboard for: {module_id}"))
    module_plot_list <- list()
    
    #####################
    ### Main Plotting ###
    #####################
    
    # Rectangle
    rect_color <- wgcna_color_tbl %>% filter(assigned_module == module_id) %>% pull(wgcna_color)
    rect_plot <- ggplot() + geom_rect(aes(xmin = 1, xmax = 1.5, ymin = 10, ymax = 15),
                                  fill = rect_color, alpha = 1, color = "black") + theme_void()

    # Top 25 module genes by kME
    one_mod_kme_tbl <- big_mod_tbl %>%
                        filter(assigned_module == module_id) %>%
                        arrange(desc(kME)) %>%
                        slice_head(n = 25) %>%
                        arrange(kME) %>%
                        mutate(rare_hit = if_else(gene_id %in% rare_var_top_genes$gene_id, 1, 0))
    one_mod_kme_tbl$gene_name <- make.names(one_mod_kme_tbl$gene_name,unique=T)
    one_mod_kme_tbl$gene_name <- as_factor(one_mod_kme_tbl$gene_name)

    kme_plot <- ggplot(one_mod_kme_tbl, aes(x = kME, y = gene_name, fill = factor(rare_hit))) +
        geom_col() +
        geom_text(aes(label = format(round(kME, 2), nsmall = 2)), hjust = -0.1) +
        coord_cartesian(xlim = c(0.7, 1)) +
        scale_x_continuous(expand = c(0,0), limits=c(0,1)) +
        theme_classic() + xlab("kME") + ylab("") +
        theme(axis.text = element_text(size = 13), legend.position="none") +
        theme(plot.margin = margin(t = 100, r = 50, b = 50, l = 0, unit = "pt")) +
        scale_fill_manual(values=c("#9BAA88", "#535F44"))

    # Overlap top other modules
    one_mod_compare_tbl <- overlap_tbl %>%
                            filter(module == module_id) %>%
                            filter(sig == 1) %>%
                            arrange(desc(or)) %>%
                            slice_head(n = 10) %>%
                            arrange(or) %>%
                            mutate(in_prev = if_else(grepl("werling_|walker_|li_", compare_against), 1, 0))
    one_mod_compare_tbl$compare_against <- factor(one_mod_compare_tbl$compare_against, unique(one_mod_compare_tbl$compare_against))
    compare_plot <- ggplot(one_mod_compare_tbl, aes(x = or, y = compare_against, fill = factor(in_prev))) +
                        geom_col() +
                        geom_text(aes(label = format(round(or, 1), nsmall = 1)), hjust = -0.1) +
                        coord_cartesian(xlim = c(0, max(one_mod_compare_tbl$or) + 40)) +
                        scale_x_continuous(expand = c(0,0), limits=c(0,100000)) +
                        theme_classic() + xlab("Overlap OR") + ylab("") +
                        theme(axis.text = element_text(size = 13), legend.position="none") +
                        theme(plot.margin = margin(t = 100, r = 50, b = 50, l = 0, unit = "pt")) +
                        scale_fill_manual(values=c("#D897A4", "#175676"))

    # Cell type results
    one_mod_cell_tbl <- enrich_tbl %>%
                            filter(enrich_method == "cell_type_ORA") %>%
                            filter(module == module_id) %>%
                            arrange(desc(enrich_group))
    one_mod_cell_tbl$enrich_group <- factor(one_mod_cell_tbl$enrich_group, unique(one_mod_cell_tbl$enrich_group))
    cell_plot <- ggplot(one_mod_cell_tbl, aes(x = -log10(p), y = enrich_group, fill = factor(sig))) +
                    geom_col(position=position_dodge(), width = 0.8) +
                    scale_x_continuous(expand = c(0,0), limits=c(0,
                                                                 max(50, max(-log10(one_mod_cell_tbl$p)) + 10)
                                                                )) +
                    theme_classic() + xlab("-log10(P)") + ylab("Cell Type (Polioudakis)") +
                    theme(axis.text = element_text(size = 12), legend.position="none") +
                    theme(plot.margin = margin(t = 100, r = 50, b = 50, l = 0, unit = "pt")) +
                    scale_fill_manual(values=c("0" = "#F7F3F7",
                                               "1" = "#885A89"))

    # Common variation
    one_mod_ldsc_tbl <- enrich_tbl %>%
                            filter(enrich_method == "gwas_LDSC") %>%
                            filter(module == module_id) %>%
                            mutate(method = paste0("LDSC", sig))
    one_mod_magma_tbl <- enrich_tbl %>%
                            filter(enrich_method == "gwas_MAGMA") %>%
                            filter(module == module_id) %>%
                            mutate(method = paste0("MAGMA", sig))
    combined <- rbind(one_mod_ldsc_tbl, one_mod_magma_tbl)
    combined <- combined %>% arrange(desc(enrich_group))
    combined$enrich_group <- factor(combined$enrich_group, levels = unique(combined$enrich_group))
    common_plot <- ggplot(combined, aes(x = -log10(p), y = enrich_group, fill = factor(method,
                        levels = c("LDSC0","LDSC1","MAGMA0","MAGMA1")))) +
                    geom_col(position=position_dodge(), width = 0.8) +
                    scale_x_continuous(expand = c(0,0), limits=c(0,20)) +
                    theme_classic() + xlab("-log10(P)") + ylab("GWAS") +
                    theme(axis.text = element_text(size = 12), legend.position="none") +
                    theme(plot.margin = margin(t = 100, r = 50, b = 50, l = 0, unit = "pt")) +
                    scale_fill_manual(values=c("LDSC0" = "#EFF4FB",
                                                "LDSC1" = "#6DA0D9",
                                                "MAGMA0" = "#FCEDED",
                                                "MAGMA1" = "#E66060"))

    # Rare variation
    one_mod_rare_tbl <- enrich_tbl %>%
                            filter(enrich_method == "rare_logit") %>%
                            filter(module == module_id) %>%
                            mutate(sig = if_else(or > 1, sig, 0)) %>%
                            arrange(desc(enrich_group)) %>%
                            mutate(label = paste0(format(round(or, 2), nsmall = 2), if_else(sig == 1, "*", "")))
    one_mod_rare_tbl$enrich_group <- factor(one_mod_rare_tbl$enrich_group, levels = unique(one_mod_rare_tbl$enrich_group))
    rare_plot <- ggplot(one_mod_rare_tbl, aes(x = -log10(p), y = enrich_group, fill = factor(sig))) +
                    geom_col(position=position_dodge(), width = 0.8) +
                    geom_text(aes(label = label), hjust = -0.1) +
                    scale_x_continuous(expand = c(0,0), limits=c(0,
                                                                 max(20, max(-log10(one_mod_rare_tbl$p)) + 10)
                                                                )) +
                    theme_classic() + xlab("-log10(P)\n# is OR") + ylab("Study") +
                    theme(axis.text = element_text(size = 12), legend.position="none") +
                    theme(plot.margin = margin(t = 100, r = 50, b = 50, l = 0, unit = "pt")) +
                    scale_fill_manual(values=c("0" = "#FFEBD6",
                                               "1" = "#CC6900"))

    # GO enrichment
    go_genes <- big_mod_tbl %>%
                    filter(assigned_module == module_id) %>%
                    group_by(gene_id) %>%
                    filter(kME == max(kME)) %>%
                    ungroup %>%
                    filter(kME > 0) %>%
                    arrange(desc(kME)) %>%
                    pull(gene_id) %>%
                    unique

    gostres <- gprofiler2::gost(query = go_genes, 
                    organism = "hsapiens",
                    domain_scope = "custom",
                    custom_bg = go_bg,
                    ordered_query = TRUE,
                    significant = FALSE)
    
    result_tbl <- gostres$result %>%
        filter(significant == TRUE) %>%
        select(source, term_size, intersection_size, p_value, term_id, term_name) %>%
        filter(term_size < 2000)

    # Take the top results for important categories
    top_terms <- result_tbl %>%
                filter(source %in% c("GO:BP","GO:MF","GO:CC","KEGG","REAC")) %>%
                group_by(source) %>%
                arrange(p_value, term_size) %>%
                slice(1:3) %>%
                ungroup %>%
                arrange(p_value) %>%
                select(term_id) %>%
                pull
    
    if (is_empty(top_terms)) {
        top_terms = gostres$result %>%
            filter(p_value < 1) %>%
            slice_head(n = 5) %>%
            pull(term_id)
    }

    # The gostplot function automatically plots the results, suppress that with pdf(file=NULL)
    pdf(file = NULL)
    gplot <- gprofiler2::gostplot(gostres, capped = TRUE, interactive = FALSE)
    go_plot <- gprofiler2::publish_gostplot(gplot, highlight_terms = top_terms) +
                theme(plot.margin = margin(t = 100, r = 50, b = 50, l = 0, unit = "pt"))
    dev.off()

    ##########################
    ### Assemble Dashboard ###
    ##########################
    dashboard <- arrangeGrob(rect_plot, kme_plot, compare_plot, cell_plot, common_plot, rare_plot, go_plot,
                            ncol = 7, nrow = 1, widths=c(1, 15, 15, 15, 15, 15, 35))

    mod_size <- big_mod_tbl %>%
                    filter(assigned_module == module_id) %>%
                    count %>%
                    pull
    
    full_dendro_name <- wgcna_color_tbl %>% filter(assigned_module == module_id) %>% pull(full_dendro_name)
    dash_ggplot <- ggplotify::as.ggplot(dashboard) + ylab(paste0(full_dendro_name, " [", mod_size, "]")) +
                                        theme(axis.title.y = element_text(vjust = 0.5, angle = 90, size = 12,
                                                                          face = "bold", color = "black",
                                                                          margin = margin(0,-1.35,0,0.7, unit = 'cm')))
    module_plot_list <- append(module_plot_list, list(dash_ggplot))
    
    print(glue("[INFO] Saving module dashboard for {module_id} as: {full_dendro_name}.pdf"))
    
    # Write to PDF
    ggsave(file.path("dashboards", glue("{full_dendro_name}.pdf")),
           plot = marrangeGrob(module_plot_list, nrow = 1, ncol = 1, top = NULL),
           height = 10, width = 40, dpi = "retina", units = "in", device='pdf')
}

[INFO] Generating module dashboard for: M1
[INFO] Saving module dashboard for M1 as: M1-tr2.pdf
[INFO] Generating module dashboard for: M2
[INFO] Saving module dashboard for M2 as: M2-gene.pdf
[INFO] Generating module dashboard for: M3
[INFO] Saving module dashboard for M3 as: M3-xx.pdf
[INFO] Generating module dashboard for: M4
[INFO] Saving module dashboard for M4 as: M4-xy.pdf
[INFO] Generating module dashboard for: M5
[INFO] Saving module dashboard for M5 as: M5-iso.pdf
[INFO] Generating module dashboard for: M6
[INFO] Saving module dashboard for M6 as: M6-iso.pdf
[INFO] Generating module dashboard for: M7
[INFO] Saving module dashboard for M7 as: M7-tr1.pdf
[INFO] Generating module dashboard for: M8
[INFO] Saving module dashboard for M8 as: M8-xy.pdf
[INFO] Generating module dashboard for: M9
[INFO] Saving module dashboard for M9 as: M9-iso.pdf
[INFO] Generating module dashboard for: M10
[INFO] Saving module dashboard for M10 as: M10-iso.pdf
[INFO] Generating module dashboard for:

None of the term IDs in the 'highlight_terms' was found from the results.



[INFO] Saving module dashboard for M13 as: M13-xy.pdf
[INFO] Generating module dashboard for: M14
[INFO] Saving module dashboard for M14 as: M14-xx.pdf
[INFO] Generating module dashboard for: M15
[INFO] Saving module dashboard for M15 as: M15-gene.pdf
[INFO] Generating module dashboard for: M16
[INFO] Saving module dashboard for M16 as: M16-xy.pdf
[INFO] Generating module dashboard for: M17
[INFO] Saving module dashboard for M17 as: M17-tr1.pdf
[INFO] Generating module dashboard for: M18
[INFO] Saving module dashboard for M18 as: M18-xx.pdf
[INFO] Generating module dashboard for: M19
[INFO] Saving module dashboard for M19 as: M19-gene.pdf
[INFO] Generating module dashboard for: M20
[INFO] Saving module dashboard for M20 as: M20-tr2.pdf
[INFO] Generating module dashboard for: M21
[INFO] Saving module dashboard for M21 as: M21-xy.pdf
[INFO] Generating module dashboard for: M22
[INFO] Saving module dashboard for M22 as: M22-iso.pdf
[INFO] Generating module dashboard for: M23
[INFO] Saving

Warning message in max(one_mod_compare_tbl$or):
“no non-missing arguments to max; returning -Inf”


[INFO] Saving module dashboard for M29 as: M29-iso.pdf
[INFO] Generating module dashboard for: M30
[INFO] Saving module dashboard for M30 as: M30-gene.pdf
[INFO] Generating module dashboard for: M31
[INFO] Saving module dashboard for M31 as: M31-iso.pdf
[INFO] Generating module dashboard for: M32
[INFO] Saving module dashboard for M32 as: M32-tr1.pdf
[INFO] Generating module dashboard for: M33
[INFO] Saving module dashboard for M33 as: M33-tr1.pdf
[INFO] Generating module dashboard for: M34
[INFO] Saving module dashboard for M34 as: M34-tr2.pdf
[INFO] Generating module dashboard for: M35
[INFO] Saving module dashboard for M35 as: M35-iso.pdf
[INFO] Generating module dashboard for: M36
[INFO] Saving module dashboard for M36 as: M36-xy.pdf
[INFO] Generating module dashboard for: M37
[INFO] Saving module dashboard for M37 as: M37-gene.pdf
[INFO] Generating module dashboard for: M38
[INFO] Saving module dashboard for M38 as: M38-xx.pdf
[INFO] Generating module dashboard for: M39
[INFO] Sav

In [29]:
# Rename files for filesystem ordering
old_names <- paste0(wgcna_color_tbl$full_dendro_name,".pdf")
old_names

new_names <- sprintf("%03d", as.integer(str_extract(wgcna_color_tbl$full_dendro_name, "[0-9]+")))
new_names <- str_replace(old_names, "[0-9]+", new_names)
new_names

[1] "M1-tr2.pdf"    "M10-iso.pdf"   "M100-xx.pdf"   "M101-xy.pdf"  
  [5] "M102-gene.pdf" "M103-tr1.pdf"  "M104-tr2.pdf"  "M105-iso.pdf" 
  [9] "M106-tr2.pdf"  "M107-tr1.pdf"  "M108-xx.pdf"   "M109-gene.pdf"
 [13] "M11-iso.pdf"   "M110-xy.pdf"   "M111-tr1.pdf"  "M112-xx.pdf"  
 [17] "M113-xy.pdf"   "M114-gene.pdf" "M115-tr2.pdf"  "M116-tr1.pdf" 
 [21] "M117-iso.pdf"  "M118-tr2.pdf"  "M119-xy.pdf"   "M12-gene.pdf" 
 [25] "M120-iso.pdf"  "M121-tr1.pdf"  "M122-iso.pdf"  "M123-iso.pdf" 
 [29] "M124-iso.pdf"  "M13-xy.pdf"    "M14-xx.pdf"    "M15-gene.pdf" 
 [33] "M16-xy.pdf"    "M17-tr1.pdf"   "M18-xx.pdf"    "M19-gene.pdf" 
 [37] "M2-gene.pdf"   "M20-tr2.pdf"   "M21-xy.pdf"    "M22-iso.pdf"  
 [41] "M23-tr1.pdf"   "M24-gene.pdf"  "M25-tr2.pdf"   "M26-xx.pdf"   
 [45] "M27-tr1.pdf"   "M28-tr1.pdf"   "M29-iso.pdf"   "M3-xx.pdf"    
 [49] "M30-gene.pdf"  "M31-iso.pdf"   "M32-tr1.pdf"   "M33-tr1.pdf"  
 [53] "M34-tr2.pdf"   "M35-iso.pdf"   "M36-xy.pdf"    "M37-gene.pdf" 
 [57] "M38-xx.pdf"    "M39-tr1.pdf"   "M4-xy.pdf"     "M40-iso.pdf"  
 [61] "M41-tr1.pdf"   "M42-xx.pdf"    "M43-iso.pdf"   "M44-tr1.pdf"  
 [65] "M45-iso.pdf"   "M46-tr2.pdf"   "M47-xy.pdf"    "M48-tr1.pdf"  
 [69] "M49-gene.pdf"  "M5-iso.pdf"    "M50-xx.pdf"    "M51-iso.pdf"  
 [73] "M52-tr2.pdf"   "M53-xy.pdf"    "M54-iso.pdf"   "M55-gene.pdf" 
 [77] "M56-xx.pdf"    "M57-xx.pdf"    "M58-gene.pdf"  "M59-iso.pdf"  
 [81] "M6-iso.pdf"    "M60-tr2.pdf"   "M61-tr1.pdf"   "M62-xy.pdf"   
 [85] "M63-gene.pdf"  "M64-xx.pdf"    "M65-iso.pdf"   "M66-gene.pdf" 
 [89] "M67-xx.pdf"    "M68-iso.pdf"   "M69-gene.pdf"  "M7-tr1.pdf"   
 [93] "M70-tr1.pdf"   "M71-xy.pdf"    "M72-tr2.pdf"   "M73-xx.pdf"   
 [97] "M74-tr1.pdf"   "M75-iso.pdf"   "M76-iso.pdf"   "M77-iso.pdf"  
[101] "M78-gene.pdf"  "M79-xx.pdf"    "M8-xy.pdf"     "M80-tr2.pdf"  
[105] "M81-xy.pdf"    "M82-iso.pdf"   "M83-tr2.pdf"   "M84-gene.pdf" 
[109] "M85-xx.pdf"    "M86-gene.pdf"  "M87-tr2.pdf"   "M88-xx.pdf"   
[113] "M89-iso.pdf"   "M9-iso.pdf"    "M90-tr1.pdf"   "M91-xy.pdf"   
[117] "M92-iso.pdf"   "M93-tr1.pdf"   "M94-xy.pdf"    "M95-xy.pdf"   
[121] "M96-tr1.pdf"   "M97-gene.pdf"  "M98-xx.pdf"    "M99-iso.pdf"

[1] "M001-tr2.pdf"  "M010-iso.pdf"  "M100-xx.pdf"   "M101-xy.pdf"  
  [5] "M102-gene.pdf" "M103-tr1.pdf"  "M104-tr2.pdf"  "M105-iso.pdf" 
  [9] "M106-tr2.pdf"  "M107-tr1.pdf"  "M108-xx.pdf"   "M109-gene.pdf"
 [13] "M011-iso.pdf"  "M110-xy.pdf"   "M111-tr1.pdf"  "M112-xx.pdf"  
 [17] "M113-xy.pdf"   "M114-gene.pdf" "M115-tr2.pdf"  "M116-tr1.pdf" 
 [21] "M117-iso.pdf"  "M118-tr2.pdf"  "M119-xy.pdf"   "M012-gene.pdf"
 [25] "M120-iso.pdf"  "M121-tr1.pdf"  "M122-iso.pdf"  "M123-iso.pdf" 
 [29] "M124-iso.pdf"  "M013-xy.pdf"   "M014-xx.pdf"   "M015-gene.pdf"
 [33] "M016-xy.pdf"   "M017-tr1.pdf"  "M018-xx.pdf"   "M019-gene.pdf"
 [37] "M002-gene.pdf" "M020-tr2.pdf"  "M021-xy.pdf"   "M022-iso.pdf" 
 [41] "M023-tr1.pdf"  "M024-gene.pdf" "M025-tr2.pdf"  "M026-xx.pdf"  
 [45] "M027-tr1.pdf"  "M028-tr1.pdf"  "M029-iso.pdf"  "M003-xx.pdf"  
 [49] "M030-gene.pdf" "M031-iso.pdf"  "M032-tr1.pdf"  "M033-tr1.pdf" 
 [53] "M034-tr2.pdf"  "M035-iso.pdf"  "M036-xy.pdf"   "M037-gene.pdf"
 [57] "M038-xx.pdf"   "M039-tr1.pdf"  "M004-xy.pdf"   "M040-iso.pdf" 
 [61] "M041-tr1.pdf"  "M042-xx.pdf"   "M043-iso.pdf"  "M044-tr1.pdf" 
 [65] "M045-iso.pdf"  "M046-tr2.pdf"  "M047-xy.pdf"   "M048-tr1.pdf" 
 [69] "M049-gene.pdf" "M005-iso.pdf"  "M050-xx.pdf"   "M051-iso.pdf" 
 [73] "M052-tr2.pdf"  "M053-xy.pdf"   "M054-iso.pdf"  "M055-gene.pdf"
 [77] "M056-xx.pdf"   "M057-xx.pdf"   "M058-gene.pdf" "M059-iso.pdf" 
 [81] "M006-iso.pdf"  "M060-tr2.pdf"  "M061-tr1.pdf"  "M062-xy.pdf"  
 [85] "M063-gene.pdf" "M064-xx.pdf"   "M065-iso.pdf"  "M066-gene.pdf"
 [89] "M067-xx.pdf"   "M068-iso.pdf"  "M069-gene.pdf" "M007-tr1.pdf" 
 [93] "M070-tr1.pdf"  "M071-xy.pdf"   "M072-tr2.pdf"  "M073-xx.pdf"  
 [97] "M074-tr1.pdf"  "M075-iso.pdf"  "M076-iso.pdf"  "M077-iso.pdf" 
[101] "M078-gene.pdf" "M079-xx.pdf"   "M008-xy.pdf"   "M080-tr2.pdf" 
[105] "M081-xy.pdf"   "M082-iso.pdf"  "M083-tr2.pdf"  "M084-gene.pdf"
[109] "M085-xx.pdf"   "M086-gene.pdf" "M087-tr2.pdf"  "M088-xx.pdf"  
[113] "M089-iso.pdf"  "M009-iso.pdf"  "M090-tr1.pdf"  "M091-xy.pdf"  
[117] "M092-iso.pdf"  "M093-tr1.pdf"  "M094-xy.pdf"   "M095-xy.pdf"  
[121] "M096-tr1.pdf"  "M097-gene.pdf" "M098-xx.pdf"   "M099-iso.pdf"

In [30]:
file.rename(old_names,new_names)

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [16] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [31] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [46] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [61] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [76] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [91] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[106] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[121] TRUE TRUE TRUE TRUE